# Análisis visual interactivo con Folium

Importamos las bibliotecas necesarias

In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import requests
import io

Primero agregamos la ubicación de cada sitio en un mapa usando las coordenadas de latitud y longitud del sitio.

El siguiente conjunto de datos con el nombre spacex_launch_geo.csv es un conjunto de datos ampliado con latitud y longitud agregadas para cada sitio.

In [2]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)

if response.status_code == 200:
    spacex_csv_file = response.content
    spacex_df = pd.read_csv(io.BytesIO(spacex_csv_file))
    print(spacex_df.head())
else:
    print(f"Error al descargar el archivo: {response.status_code}")

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

Ahora se pueden revisar las coordenadas de cada sitio.

In [3]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Visualizaremos las coordenadas anteriores para ver las ubicaciones marcándolas en el mapa. Primero debemos crear un objeto de mapa Folium, con una ubicación central inicial que será el Centro Espacial Johnson de la NASA en Houston, Texas.

In [4]:
# La ubicación de inicio es el Centro Espacial Johnson de la NASA
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Podríamos usar folium.Circle para agregar un área de círculo resaltada con una etiqueta de texto en una coordenada específica. Por ejemplo,

In [5]:
# Creamos un círculo azul en las coordenadas del Centro Espacial Johnson de la NASA con una etiqueta emergente que muestra su nombre
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Creamos un círculo azul en las coordenadas del Centro Espacial Johnson de la NASA con un ícono que muestra su nombre
marker = folium.map.Marker(
    nasa_coordinate,
    # Creamos un icono como etiqueta de texto
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Ahora, agreguemos un círculo para cada sitio de lanzamiento en el marco de datos launch_sites

In [6]:
# Iteramos sobre cada fila en el marco de datos launch_sites_df
for index, row in launch_sites_df.iterrows():
    # Extraemos la latitud y la longitud de cada sitio de lanzamiento
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    
    # Generamos un círculo en las coordenadas del sitio de lanzamiento con una etiqueta emergente que muestra su nombre
    circle = folium.Circle([lat, lon], radius=1000, color='#3186cc', fill=True).add_child(folium.Popup(launch_site))
    
    # Generamos un marcador en las coordenadas del sitio de lanzamiento con un icono que muestra su nombre
    marker = folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#3186cc;"><b>%s</b></div>' % launch_site,
            )
        )
    
    # Agregamos el círculo y el marcador al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostramos el mapa con los círculos y marcadores de los sitios de lanzamiento
site_map

A continuación, se intentará mejorar el mapa agregando los resultados de lanzamiento de cada sitio y veamos qué sitios tienen tasas de éxito altas. Es importante recordar que el marco de datos spacex_df tiene registros de lanzamiento detallados y la columna de clase indica si este lanzamiento fue exitoso o no.

In [7]:
spacex_df.tail(3)

,Launch Site,Lat,Long,class
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


A continuación, crearemos marcadores para todos los registros de lanzamiento. Si un lanzamiento fue exitoso `(class=1)`, entonces usamos un marcador verde y si un lanzamiento falló, usamos un marcador rojo `(class=0)`

Hay que tener en cuenta que un lanzamiento solo se produce en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los grupos de marcadores pueden ser una buena forma de simplificar un mapa que contiene muchos marcadores que tienen la misma coordenada.

Primero, creemos un objeto `MarkerCluster`

In [8]:
marker_cluster = MarkerCluster()

Creamos una nueva columna en el marco de datos `spacex_df` llamada `marker_color` para almacenar los colores de los marcadores según el valor de `class`

In [9]:
# Definimos una función para determinar el color del marcador
def get_marker_color(launch_class):
    if launch_class == 1:
        return 'green'
    else:
        return 'red'

# Iteramos sobre cada fila en el marco de datos launch_sites_df
for index, row in spacex_df.iterrows():
    # Extraemos la latitud, longitud, nombre del sitio de lanzamiento y clase
    lat = row['Lat']
    lon = row['Long']
    launch_site = row['Launch Site']
    launch_class = row['class']
    
    # Obtenemos el color del marcador basado en el valor de la clase
    marker_color = get_marker_color(launch_class)
    
    # Creamos un círculo en las coordenadas del sitio de lanzamiento con una etiqueta emergente que muestra su nombre
    circle = folium.Circle([lat, lon], radius=1000, color=marker_color, fill=True).add_child(folium.Popup(launch_site))
    
    # Creamos un marcador en las coordenadas del sitio de lanzamiento con un icono que muestra su nombre
    marker = folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:{marker_color};"><b>{launch_site}</b></div>',
            )
        )
    
    # Agregamos el círculo y el marcador al mapa
    site_map.add_child(circle)
    site_map.add_child(marker)

# Mostramos el mapa con los círculos y marcadores de los sitios de lanzamiento
site_map

Para cada resultado de lanzamiento en el marco de datos `spacex_df`, agregaremos un `folium.Marker` a `marker_cluster`

A partir de los marcadores etiquetados por color en los grupos de marcadores, debería se pueden identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.

A continuación, debemos explorar y analizar la proximidad de los sitios de lanzamiento.

Primero, agreguamos una `Posición del mouse` en el mapa para obtener las coordenadas del mouse sobre un punto del mapa. De esta manera, mientras se explora el mapa se pueden encontrar fácilmente las coordenadas de cualquier punto de interés.

In [10]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Función para determinar la distancia entre dos puntos

In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la Tierra en km.
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Marcamos un punto en la costa más cercana usando MousePosition y calculamos la distancia entre el punto de la costa y el sitio de lanzamiento.

In [13]:
# Coordenadas del sitio de lanzamiento (ejemplo)
launch_site_coords = [29.56092, -95.07993]

# Coordenadas del punto en la costa más cercana (ejemplo)
coast_point_coords = [29.55864, -95.07407]

# Agregar un marcador en el sitio de lanzamiento
folium.Marker(location=launch_site_coords, popup="Sitio de Lanzamiento").add_to(site_map)

# Agregar un marcador en el punto de la costa más cercana
folium.Marker(location=coast_point_coords, popup="Punto en la Costa Más Cercana").add_to(site_map)

# Calcular la distancia entre el sitio de lanzamiento y el punto en la costa
distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], coast_point_coords[0], coast_point_coords[1])

# Mostrar la distancia
print("Distancia entre el sitio de lanzamiento y el punto en la costa más cercana:", distance, "km")

# Mostrar el mapa
site_map

Distancia entre el sitio de lanzamiento y el punto en la costa más cercana: 0.6211025531566161 km


In [14]:
coordinate = [29.55864, -95.07407]

distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
       icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
        )
    )

site_map.add_child(circle)
site_map.add_child(marker)

site_map

Dibujamos una polilínea entre un sitio de lanzamiento y el punto de la costa seleccionado

In [15]:
coordinates = [launch_site_coords, coast_point_coords]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

TODO: De manera similar, se puede dibujar una línea entre un sitio de lanzamiento y su ciudad, ferrocarril, autopista, etc. más cercana. Primero se debe usar la posición del mouse para encontrar sus coordenadas en el mapa.